In [1]:
from carbon import CarbonSimulatorUI, __version__, __date__
print(f"Carbon Version v{__version__} ({__date__})", )

Carbon Version v1.5-beta0 (6/Dec/2022)


# Carbon Simulation - Demo 1-1

In this demo we look at **single-curve unidirectional liquidity with a zero width range**

Set up the similation, with the pair USDCETH as default

In [2]:
Sim = CarbonSimulatorUI(pair="ETH/USDC", verbose=False, raiseonerror=False)
Sim

CarbonSimulatorUI(<0 orders, 0 trades>, pair='ETH/USDC', mm='exact', xf=True)

In [3]:
help(Sim.add_order)

Help on method add_sellorder in module carbon.simulators.carbon_simulator:

add_sellorder(tkn: str, amt: Any, p_start: Any, p_end: Any, pair: str = None) -> Dict[str, Any] method of carbon.simulators.carbon_simulator.CarbonSimulatorUI instance
    adds a sell order for tkn
    
    :tkn:           the token that is being added to the position, eg "ETH"; it is the token being sold*
    :amt:           the amount of `tkn` that is added to the position
    :p_start:       the start* of the range, quoted in the currency of the pair
    :p_end:         ditto end*
    :pair:          the token pair to which the position corresponds, eg "ETHUSD"*
    
    *p_start, p_end are interchangeable, the code deals with sorting them correctly,
    albeit with a warning message if they were in the wrong order



We remind ourselves the price convention for `ETHUSDC` (`ETH` or `USDC` has not semantic meaning here other than allowing to split the pair into its constituent tokens)

In [4]:
Sim.price_convention("ETH/USDC", "ETH"), Sim.price_convention("ETH/USDC", "USDC")

('USDC per ETH', 'USDC per ETH')

We assert that we cannot add an unrelated position to this pair, either using an existing token like LINK, or a token like DNE (does not exist).

In [5]:
Sim.add_order("LINK", 100, 2000, 2000)

{'success': False,
 'error': "('Token not part of pair', 'LINK', CarbonPair(slashpair='ETH/USDC', tknb='ETH', tknq='USDC'), None)",
 'exception': ValueError('Token not part of pair',
            'LINK',
            CarbonPair(slashpair='ETH/USDC', tknb='ETH', tknq='USDC'),
            None)}

In [6]:
Sim.add_order("DNE", 100, 2000, 2000)

{'success': False,
 'error': "('Token not part of pair', 'DNE', CarbonPair(slashpair='ETH/USDC', tknb='ETH', tknq='USDC'), None)",
 'exception': ValueError('Token not part of pair',
            'DNE',
            CarbonPair(slashpair='ETH/USDC', tknb='ETH', tknq='USDC'),
            None)}

We set up a single curve where the AMM sells ETH against USDC at 2000

In [7]:
Sim.add_order("ETH", 100, 2000, 2000)["orders"]

,id,pair,tkn,y_int,y,y_unit,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,ETH,100.0,100.0,ETH,2000.0,2000.0,2000.0,USDC per ETH,0


In [8]:
Sim.state()["orders"]

,id,pair,tkn,y_int,y,y_unit,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,ETH,100.0,100.0,ETH,2000.0,2000.0,2000.0,USDC per ETH,0


We are trying to make the AMM buy ETH. That fails because it does not have any USDC.

In [9]:
Sim.amm_buys("ETH", 10)

{'success': False,
 'error': 'token USDC has no non-empty liquidity positions',
 'exception': ValueError('token USDC has no non-empty liquidity positions')}

Now we are trying to make the AMM sell more ETH than it has. That also fails.

In [10]:
Sim.amm_sells("ETH", 101)

{'success': False,
 'error': 'Insufficient liquidity across all user positions to support this trade.',
 'exception': AssertionError('Insufficient liquidity across all user positions to support this trade.')}

However, the AMM can sell 10 ETH, at a price of 2000.

In [11]:
Sim.amm_sells("ETH", 10)["trades"]

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #0,False,True,None,10.0,ETH,20000.0,USDC,ETHUSDC,0,1,2000.0,USDC per ETH
0,0,0,A,AMM sells 10ETH buys 20000USDC,True,True,None,10.0,ETH,20000.0,USDC,ETHUSDC,[0],1,2000.0,USDC per ETH


In [12]:
Sim.state()["orders"]

,id,pair,tkn,y_int,y,y_unit,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,ETH,100.0,90.0,ETH,2000.0,2000.0,2000.0,USDC per ETH,0


We've sold 10 above, so can we sell 90+epsilon? Obviously not

In [13]:
Sim.amm_sells("ETH", 90.0000001)

{'success': False,
 'error': 'Insufficient liquidity across all user positions to support this trade.',
 'exception': AssertionError('Insufficient liquidity across all user positions to support this trade.')}

However, it can sell 90

In [14]:
Sim.amm_sells("ETH", 90)["trades"]

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,1.0,1,0,route #0,False,True,None,90.0,ETH,180000.0,USDC,ETHUSDC,0,1,2000.0,USDC per ETH
0,1,1,A,AMM sells 90ETH buys 180000USDC,True,True,None,90.0,ETH,180000.0,USDC,ETHUSDC,[0],1,2000.0,USDC per ETH


In [15]:
Sim.state()["orders"]

,id,pair,tkn,y_int,y,y_unit,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,ETH,100.0,0.0,ETH,2000.0,2000.0,2000.0,USDC per ETH,0


In [16]:
Sim.state()["trades"]

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #0,False,True,None,10.0,ETH,20000.0,USDC,ETHUSDC,0,1,2000.0,USDC per ETH
0,0,0,A,AMM sells 10ETH buys 20000USDC,True,True,None,10.0,ETH,20000.0,USDC,ETHUSDC,[0],1,2000.0,USDC per ETH
0,1.0,1,0,route #0,False,True,None,90.0,ETH,180000.0,USDC,ETHUSDC,0,1,2000.0,USDC per ETH
0,1,1,A,AMM sells 90ETH buys 180000USDC,True,True,None,90.0,ETH,180000.0,USDC,ETHUSDC,[0],1,2000.0,USDC per ETH


Can we now sell epsilon?

In [17]:
Sim.amm_sells("ETH", 0.0000001)

{'success': False,
 'error': 'token ETH has no non-empty liquidity positions',
 'exception': ValueError('token ETH has no non-empty liquidity positions')}

In [18]:
Sim.amm_sells("ETH", 0)

{'success': False,
 'error': 'token ETH has no non-empty liquidity positions',
 'exception': ValueError('token ETH has no non-empty liquidity positions')}